## Imports

In [1]:
import datetime
import os

In [2]:
from sklearn.model_selection import StratifiedKFold

In [3]:
from keras import backend as K
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


## Config

In [4]:
feature_list_id = 'oofp_nn_concat_dense_1'

In [5]:
RANDOM_SEED = 42

In [6]:
np.random.seed(RANDOM_SEED)

In [7]:
data_folder = os.path.abspath(os.path.join(os.curdir, os.pardir, 'data')) + os.path.sep
aux_data_folder = os.path.join(data_folder, 'aux') + os.path.sep
preproc_data_folder = os.path.join(data_folder, 'preproc') + os.path.sep
features_data_folder = os.path.join(data_folder, 'features') + os.path.sep
submissions_data_folder = os.path.join(data_folder, 'submissions') + os.path.sep

## Read Data

In [8]:
embedding_matrix = load(aux_data_folder + 'embedding_weights_fasttext_filtered_no_stopwords.pickle')

In [9]:
X_train_q1 = load(features_data_folder + 'X_train_nn_fasttext_q1_filtered_no_stopwords.pickle')
X_train_q2 = load(features_data_folder + 'X_train_nn_fasttext_q2_filtered_no_stopwords.pickle')

In [10]:
X_train = np.stack((X_train_q1, X_train_q2), axis=1)

In [11]:
y_train = load(features_data_folder + 'y_train.pickle')

In [12]:
EMBEDDING_DIM = embedding_matrix.shape[-1]
VOCAB_LENGTH = embedding_matrix.shape[0] - 1
MAX_SEQUENCE_LENGTH = X_train.shape[-1]

In [13]:
print(EMBEDDING_DIM, VOCAB_LENGTH, MAX_SEQUENCE_LENGTH)

300 101441 30


## Train Models & Compute Out-of-Fold Predictions

In [14]:
def create_model():
    model_q1 = Sequential()

    model_q1.add(Embedding(
        VOCAB_LENGTH + 1,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False,
    ))

    model_q1.add(TimeDistributed(Dense(
        EMBEDDING_DIM,
        activation='relu',
    )))

    model_q1.add(Lambda(lambda x: K.max(x, axis=1), output_shape=(EMBEDDING_DIM, )))

    model_q2 = Sequential()

    model_q2.add(Embedding(
        VOCAB_LENGTH + 1,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False,
    ))

    model_q2.add(TimeDistributed(Dense(
        EMBEDDING_DIM,
        activation='relu'
    )))

    model_q2.add(Lambda(lambda x: K.max(x, axis=1), output_shape=(EMBEDDING_DIM, )))

    model = Sequential()
    model.add(Merge([model_q1, model_q2], mode='concat'))

    model.add(BatchNormalization())
    model.add(Dense(200, activation='relu'))

    model.add(BatchNormalization())
    model.add(Dense(200, activation='relu'))

    model.add(BatchNormalization())
    model.add(Dense(200, activation='relu'))

    model.add(BatchNormalization())
    model.add(Dense(200, activation='relu'))

    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        loss='binary_crossentropy', 
        optimizer='adam', 
        metrics=['accuracy']
    )

    return model

In [15]:
model_checkpoint_path = aux_data_folder + 'fold-checkpoint-' + feature_list_id + '.h5'

In [16]:
kfold = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=RANDOM_SEED
)

In [17]:
y_train_oofp = np.zeros_like(y_train, dtype='float64')

In [18]:
%%time

for fold_num, (ix_fold_train, ix_fold_val) in enumerate(kfold.split(X_train, y_train)):
    X_fold_train = X_train[ix_fold_train]
    y_fold_train = y_train[ix_fold_train]
    
    X_fold_val = X_train[ix_fold_val]
    y_fold_val = y_train[ix_fold_val]
    
    print()
    print(f'Fitting fold {fold_num + 1} of {kfold.n_splits}')
    print()
    
    model = create_model()
    model.fit(
        [X_fold_train[:, 0], X_fold_train[:, 1]], y_fold_train,
        batch_size=64,
        epochs=200,
        validation_data=([X_fold_val[:, 0], X_fold_val[:, 1]], y_fold_val),
        class_weight=keras_get_class_weights(y_fold_train),
        callbacks=[
            EarlyStopping(
                monitor='val_loss',
                min_delta=0.005,
                patience=3,
                verbose=1,
                mode='auto',
            ),
            ModelCheckpoint(
                model_checkpoint_path,
                monitor='val_loss',
                save_best_only=True,
                verbose=2,
            ),
        ],
    )
        
    # Create out-of-fold prediction.
    model.load_weights(model_checkpoint_path)
    y_pred_oofp = model.predict_proba([X_fold_val[:, 0], X_fold_val[:, 1]])[:, -1]
    
    # Remember them.
    y_train_oofp[ix_fold_val] = y_pred_oofp


Fitting fold 1 of 5



/home/yuriyguts/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Train on 323431 samples, validate on 80859 samples
Epoch 1/200
323431/323431 [==============================] - 36s - loss: 0.6532 - acc: 0.7353 - val_loss: 0.4744 - val_acc: 0.7638
Epoch 2/200
323431/323431 [==============================] - 35s - loss: 0.5645 - acc: 0.7788 - val_loss: 0.4568 - val_acc: 0.7735
Epoch 3/200
323431/323431 [==============================] - 35s - loss: 0.5076 - acc: 0.8045 - val_loss: 0.4169 - val_acc: 0.7971
Epoch 4/200
323431/323431 [==============================] - 35s - loss: 0.4631 - acc: 0.8249 - val_loss: 0.4167 - val_acc: 0.7982
Epoch 5/200
323431/323431 [==============================] - 35s - loss: 0.4261 - acc: 0.8411 - val_loss: 0.4072 - val_acc: 0.8119
Epoch 6/200
323431/323431 [==============================] - 35s - loss: 0.3923 - acc: 0.8559 - val_loss: 0.4165 - val_acc: 0.8079
Epoch 7/200
323431/323431 [==============================] - 36s - loss: 0.3640 - acc: 0.8674 - val_loss: 0.4098 - val_acc: 0.8157
Epoch 8/200
323431/323431 [=====

323432/323432 [==============================] - 31s - loss: 0.4617 - acc: 0.8266 - val_loss: 0.4157 - val_acc: 0.7969
Epoch 5/200
323432/323432 [==============================] - 33s - loss: 0.4242 - acc: 0.8419 - val_loss: 0.4058 - val_acc: 0.8080
Epoch 6/200
323432/323432 [==============================] - 32s - loss: 0.3918 - acc: 0.8567 - val_loss: 0.4014 - val_acc: 0.8131
Epoch 7/200
323432/323432 [==============================] - 32s - loss: 0.3636 - acc: 0.8683 - val_loss: 0.4010 - val_acc: 0.8136
Epoch 8/200
323432/323432 [==============================] - 31s - loss: 0.3381 - acc: 0.8786 - val_loss: 0.4154 - val_acc: 0.8188
Epoch 9/200
323432/323432 [==============================] - 31s - loss: 0.3157 - acc: 0.8877 - val_loss: 0.4218 - val_acc: 0.8186
Epoch 00008: early stopping
80256/80858 [============================>.] - ETA: 0s
Fitting fold 4 of 5

Train on 323433 samples, validate on 80857 samples
Epoch 1/200
323433/323433 [==============================] - 35s - loss

323433/323433 [==============================] - 38s - loss: 0.3365 - acc: 0.8794 - val_loss: 0.4272 - val_acc: 0.8122
Epoch 9/200
323433/323433 [==============================] - 37s - loss: 0.3145 - acc: 0.8884 - val_loss: 0.4021 - val_acc: 0.8238
Epoch 10/200
323433/323433 [==============================] - 35s - loss: 0.2951 - acc: 0.8961 - val_loss: 0.4210 - val_acc: 0.8244
Epoch 00009: early stopping
80192/80857 [============================>.] - ETA: 0sCPU times: user 38min 27s, sys: 2min 15s, total: 40min 43s
Wall time: 28min 3s


### Save Train features

In [19]:
y_train_oofp = y_train_oofp.reshape((-1, 1))

In [20]:
save(y_train_oofp, features_data_folder + f'X_train_{feature_list_id}.pickle')

## Save Test features

In [21]:
X_test_q1 = load(features_data_folder + 'X_test_nn_fasttext_q1_filtered_no_stopwords.pickle')
X_test_q2 = load(features_data_folder + 'X_test_nn_fasttext_q2_filtered_no_stopwords.pickle')

In [22]:
# It would be better to fit the model on the whole training set
# but the validation set for early stopping would be an issue.
y_test_oofp = model.predict_proba([X_test_q1, X_test_q2])[:, -1]

2344832/2345796 [============================>.] - ETA: 0s

In [23]:
y_test_oofp = y_test_oofp.reshape((-1, 1))

In [24]:
save(y_test_oofp, features_data_folder + f'X_test_{feature_list_id}.pickle')